In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import csv


class MyDataset(object):
    '''
    Class to load the dataset
    img_dir = Plots dir
    ground_dir = Ground dir where:
        column 1 = Plot name
        column 2 = Plot ground value
        column 3 = Plot GDD value
    '''
    def __init__(self, img_dir, ground_dir):
        self.img_dir = img_dir
        self.ground_dir = ground_dir
        self.df = self._get_img_df()
        self.ground_arr = self._get_ground_df()
        self.GDD_arr = self._get_GDD_df()
    
    def __len__(self):
        return len(self.df)

    def _get_img_df(self):
        img_names = os.listdir(self.img_dir)
        df = pd.DataFrame(img_names, columns=['Filename'])
        df['Global_ID'] = df.Filename.str[6:-8].astype('int') ###[Global_ID] & [6:-8] to be adjusted
        df['DOY'] = df.Filename.str[-7:-4].astype('int') ###[-7:-4] to be adjusted
        df = pd.pivot_table(df, values = 'Filename', index='Global_ID', columns='DOY', aggfunc='first').reset_index() ###[Global_ID] to be adjusted
        df= df.sort_values('Global_ID')
        return(df)

    def _get_ground_df(self):
        ground_arr =  pd.read_csv(self.ground_dir, usecols=(0,1))
        ground_arr = pd.DataFrame(ground_arr)
        ground_arr = ground_arr.sort_values(ground_arr.columns[0])
        ground_arr = ground_arr.to_numpy()
        return(ground_arr)

    def _get_GDD_df(self):
        GDD_arr = pd.read_csv(self.ground_dir, usecols=(0,3))
        GDD_arr = pd.DataFrame(GDD_arr)
        GDD_arr = GDD_arr.sort_values('Global_ID')
        GDD_arr = GDD_arr.to_numpy()
        return(GDD_arr)
        
    def _transform(self, x):
        x = np.moveaxis(x, -3, -2) ###[np.moveaxis(x, -2, -2)] to be adjusted. Dim goals: Plot:Flight:R:G:B
        ### Normalization or rescale did NOT change the results.

        # x = x.astype('float32')

        # layer = Normalization()
        # layer.adapt(x)
        # x = layer(x)
        # x = Rescaling(1.0 / 255)(x)
        # input_shape = x.shape[:2]
        # inputs = keras.Input(input_shape[0],input_shape[1])
        # x = Rescaling(1.0 / 255)(inputs)  # Rescale inputs
        # x /= 255.0

        # min = x.min()
        # max = x.max()
        # x -= min
        # x /= (max - min)
        
        return(x)

    def __getitem__(self,imgs_stacked):
        alist = [] 
        # GDDlist = [20.8595, 20.7809, 20.9571, 20.6049, 21.0528, 20.8987, 21.1707, 21.6270, 21.2946]
        # GDDdate = 0

        for column in tqdm(self.df.columns[1:]): 
            print(f"Column name:",column)
            imgs_stack = np.stack([cv2.imread(os.path.join(self.img_dir, img)) for img in self.df[column]])
            #imgs_stack = np.stack([cv2.cvtColor(cv2.imread(os.path.join(self.img_dir, img)), cv2.COLOR_BGR2HSV) for img in self.df[column]]) ### Want to change the color space? [cv2.COLOR_BGR2HSV]

            imgs_stack = self._transform(imgs_stack)
            # imgs_stack =  imgs_stack*GDDlist[GDDdate]

            alist.append([imgs_stack])
            # GDDdate += 1
        imgs_stacked = np.squeeze(np.stack(alist, axis=2))                                                                                         

        return (imgs_stacked) 
        #display(arr)

In [3]:
#%cd /content/drive/MyDrive/Soybean_maturity_Trevisan/
main_dir = '/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2022/SVREC_Mat'
img_dir = os.path.join(main_dir,'e._ClipPlots_BN_512_128')

ground_dir = os.path.join(main_dir,'d._Ground_notes', '22_SVREC_BN_ground.csv')


In [ ]:
dataset_arr =  MyDataset(img_dir, ground_dir)[object]
len(dataset_arr)

  0%|          | 0/11 [00:00<?, ?it/s]

Column name: 49
Column name: 78
Column name: 80
Column name: 83
Column name: 86
Column name: 90
Column name: 93
Column name: 96
Column name: 99
Column name: 104
Column name: 107


636

In [ ]:
dataset_arr.shape

(636, 11, 512, 128, 3)

In [ ]:
image = dataset_arr[111,5]
image.shape
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (image.min(), image.max()))

Min: 0.000, Max: 255.000


In [ ]:
image

In [4]:
dataset_df =  MyDataset(img_dir, ground_dir).df
dataset_df
#dataset_df.to_csv('C:\\temp_maturity\\UMNSoybeans_Planting_Season_2018\\df.csv')

DOY,Global_ID,49,78,80,83,86,90,93,96,99,104,107
0,22011001,SVREC_22011001_049.png,SVREC_22011001_078.png,SVREC_22011001_080.png,SVREC_22011001_083.png,SVREC_22011001_086.png,SVREC_22011001_090.png,SVREC_22011001_093.png,SVREC_22011001_096.png,SVREC_22011001_099.png,SVREC_22011001_104.png,SVREC_22011001_107.png
1,22011002,SVREC_22011002_049.png,SVREC_22011002_078.png,SVREC_22011002_080.png,SVREC_22011002_083.png,SVREC_22011002_086.png,SVREC_22011002_090.png,SVREC_22011002_093.png,SVREC_22011002_096.png,SVREC_22011002_099.png,SVREC_22011002_104.png,SVREC_22011002_107.png
2,22011003,SVREC_22011003_049.png,SVREC_22011003_078.png,SVREC_22011003_080.png,SVREC_22011003_083.png,SVREC_22011003_086.png,SVREC_22011003_090.png,SVREC_22011003_093.png,SVREC_22011003_096.png,SVREC_22011003_099.png,SVREC_22011003_104.png,SVREC_22011003_107.png
3,22011004,SVREC_22011004_049.png,SVREC_22011004_078.png,SVREC_22011004_080.png,SVREC_22011004_083.png,SVREC_22011004_086.png,SVREC_22011004_090.png,SVREC_22011004_093.png,SVREC_22011004_096.png,SVREC_22011004_099.png,SVREC_22011004_104.png,SVREC_22011004_107.png
4,22011005,SVREC_22011005_049.png,SVREC_22011005_078.png,SVREC_22011005_080.png,SVREC_22011005_083.png,SVREC_22011005_086.png,SVREC_22011005_090.png,SVREC_22011005_093.png,SVREC_22011005_096.png,SVREC_22011005_099.png,SVREC_22011005_104.png,SVREC_22011005_107.png
...,...,...,...,...,...,...,...,...,...,...,...,...
631,22063080,SVREC_22063080_049.png,SVREC_22063080_078.png,SVREC_22063080_080.png,SVREC_22063080_083.png,SVREC_22063080_086.png,SVREC_22063080_090.png,SVREC_22063080_093.png,SVREC_22063080_096.png,SVREC_22063080_099.png,SVREC_22063080_104.png,SVREC_22063080_107.png
632,22063081,SVREC_22063081_049.png,SVREC_22063081_078.png,SVREC_22063081_080.png,SVREC_22063081_083.png,SVREC_22063081_086.png,SVREC_22063081_090.png,SVREC_22063081_093.png,SVREC_22063081_096.png,SVREC_22063081_099.png,SVREC_22063081_104.png,SVREC_22063081_107.png
633,22063082,SVREC_22063082_049.png,SVREC_22063082_078.png,SVREC_22063082_080.png,SVREC_22063082_083.png,SVREC_22063082_086.png,SVREC_22063082_090.png,SVREC_22063082_093.png,SVREC_22063082_096.png,SVREC_22063082_099.png,SVREC_22063082_104.png,SVREC_22063082_107.png
634,22063083,SVREC_22063083_049.png,SVREC_22063083_078.png,SVREC_22063083_080.png,SVREC_22063083_083.png,SVREC_22063083_086.png,SVREC_22063083_090.png,SVREC_22063083_093.png,SVREC_22063083_096.png,SVREC_22063083_099.png,SVREC_22063083_104.png,SVREC_22063083_107.png


In [5]:
dataset_ground =  MyDataset(img_dir, ground_dir).ground_arr
dataset_ground

array([[22011001,       41],
       [22011002,       41],
       [22011003,       39],
       ...,
       [22063082,       41],
       [22063083,       41],
       [22063084,       42]])

In [6]:
dataset_GDD =  MyDataset(img_dir, ground_dir).GDD_arr
dataset_GDD

array([[2.20110010e+07, 1.96620000e+01],
       [2.20110020e+07, 1.96620000e+01],
       [2.20110030e+07, 1.97020408e+01],
       ...,
       [2.20630820e+07, 1.96620000e+01],
       [2.20630830e+07, 1.96620000e+01],
       [2.20630840e+07, 1.97039604e+01]])

In [ ]:
# Change the current working directory
#os.chdir('H:\\My Drive\\UMNSoybeans_Planting_Season_2018\\e._npy_data')
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2022\\SVREC_Mat\\f._numpy_data')
np.save('2022_SVREC_11fly_BN_512_128.npy', dataset_arr)  

In [ ]:
# Change the current working directory
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2022\\SVREC_Mat\\f._numpy_data')
np.save('2022_SVREC_ground_11fly_BN.npy', dataset_ground)  

In [7]:
# Change the current working directory
os.chdir('/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2022/SVREC_Mat/f._numpy_data')
np.save('2022_SVREC_GDD.npy', dataset_GDD) 